# Create Wrapper Function for WML Model

This notebook demonstrates how to create wrapper funciton for a WML deployment. The function checks whether the model usability status in OpenPages is set to "Allow" before calling the endpoint.

### Load Configuration File

In [5]:
import os
#os.environ["CONFIG_FILE"] = "wrapper_config_wml.json"
config_file = os.environ.get('CONFIG_FILE').replace("\"","")

In [6]:
import json
wrapper_config=json.load(open(f"/project_data/data_asset/{config_file}"))

In [ ]:
# Credentials for external endpoint
ml_endpoint = wrapper_config['endpoint_info']['wml_endpoint']
# WML information
deployment_space = wrapper_config['function_info']['deployment_space']
function_name = wrapper_config['function_info']['function_name']
deployment_name = wrapper_config['function_info']['deployment_name']
# OpenPages Information
op_api_key = wrapper_config['openpages_info']['op_api_key']
model_status_field = wrapper_config['openpages_info']['model_status_field']
mrm_model_id = wrapper_config['openpages_info']['mrm_model_id']
# CPD Auth
cpd_username=wrapper_config['cpd_credentials']['cpd_username']
cpd_api_key=wrapper_config['cpd_credentials']['cpd_api_key']

In [ ]:
cpd_host = "https://zen-cpd-zen.ibmcloud-roks-hhlrn-6ccd7f378ae819553d37d5f2ee142bd6-0000.mex01.containers.appdomain.cloud"
openpages_url = 'https://openpages-openpages-1-zen.ibmcloud-roks-hhlrn-6ccd7f378ae819553d37d5f2ee142bd6-0000.mex01.containers.appdomain.cloud'

## Define Wrapper Function
Below we define the wrapper function which will 1) check the usability status of the model in OpenPages and 2) call the WML model endpoint if usability status is "Allow".

In [ ]:

params = {'ml_endpoint': ml_endpoint,'cpd_username': cpd_username,'cpd_api_key': cpd_api_key,'cpd_host': cpd_host,'openpages_url':openpages_url,'model_status_field':model_status_field,'mrm_model_id': mrm_model_id,'op_api_key': op_api_key,
         'openpages_url': openpages_url}




In [ ]:
def wrapper_func(params=params):
    import requests
    import random
    import json
    from urllib.parse import urlparse
    
    cpdhost = params['cpd_host']
    cpd_username = params['cpd_username']
    cpd_api_key = params['cpd_api_key']
    cpd_host = params['cpd_host']
    op_api_key  = params['op_api_key']


    def get_model_use_status(params=params):#always two params in dict format
        data = {
                "statement": "SELECT * FROM [Model] where [Model].[Name]={0}".format(
                    '\u0027' + params['mrm_model_id'] + '\u0027'),
                "skipCount": 0
            }  
        r = requests.post("{0}/grc/api/query".format(params['openpages_url']),
                              headers={"Authorization": "Basic " + params['op_api_key']}, json=data, verify=False)
        data = r.json()
        rows = data['rows']
        model_status_key = params['model_status_field'] #something like 'MRG-Model:Status'
        res = filter(lambda x: x['name'] == model_status_key, rows[0]['fields']['field'])
        model_use_status = list(res)[0]['enumValue']['name']
        return model_use_status
    
    def getToken(cpdhost,username,apikey):
        auth_url = "{0}/icp4d-api/v1/authorize".format(cpdhost)
        auth_response = requests.post(
            auth_url,
            headers={
                "Content-Type": "application/json"
            },
            data='{"username": "'+username+'", "api_key": "'+apikey+'"}',
            verify=False)

        return auth_response.json()['token']
    

    
    def getPrediction(endpoint,payload_scoring):
        parsed_uri = urlparse(endpoint)
        #cpdhost = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)
        token=getToken(cpd_host,cpd_username,cpd_api_key)
        header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token}
        response_scoring = requests.post(endpoint, json=payload_scoring, headers=header,verify=False)
        return response_scoring.json()
    
    
    
    def score(payload):
        #prediction = getPredictionWML(params['ml_endpoint'],payload)
        if get_model_use_status(params)=="Allow":  
            prediction=getPrediction(params['ml_endpoint'],payload)
        else:
            raise ValueError("Model is not in allow state.")
        return prediction

    return score
    

## Test Function Locally

In [2]:
#input_payload = {"input_data":[{"fields":["DAY","DAY_OF_WEEK","ORIGIN_AIRPORT","DESTINATION_AIRPORT","DEPARTURE_DELAY","TAXI_OUT","DISTANCE"],"values":[[11,7,"ABQ","DFW",2,11,570],[11,7,"ABQ","DFW",2,11,570]]}]}

In [3]:
#wrapper_func()(input_payload)

In [ ]:
from ibm_watson_machine_learning import APIClient 
import os

## Store Function in Deployment Space and Deploy

In [ ]:
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
   "token": token,
   "instance_id" : "wml_local",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}
wml_client = APIClient(wml_credentials)

In [ ]:
DEFAULT_DEPLOYMENT_SPACE=deployment_space

filtered_spaces=[ space['metadata']['id'] for space in wml_client.spaces.get_details()['resources'] if space['entity']['name']== DEFAULT_DEPLOYMENT_SPACE]
if len(filtered_spaces)>0:
    space_id=filtered_spaces[0]
else:
    space_metadata = {
        wml_client.spaces.ConfigurationMetaNames.NAME: DEFAULT_DEPLOYMENT_SPACE,
        wml_client.spaces.ConfigurationMetaNames.DESCRIPTION: DEFAULT_DEPLOYMENT_SPACE,
    }
    space_details=wml_client.spaces.store(meta_props=space_metadata);
    space_id=space_details['metadata']['id']
    
print(space_id)

In [ ]:
wml_client.set.default_space(space_id)

In [ ]:
meta_data = { wml_client.repository.FunctionMetaNames.NAME : function_name,
              wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: wml_client.software_specifications.get_id_by_name('default_py3.7_opence')}
function_details = wml_client.repository.store_function(wrapper_func, meta_data)

In [ ]:
function_details

In [ ]:
metadata = {
        wml_client.deployments.ConfigurationMetaNames.NAME: deployment_name,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
model_id = function_details["metadata"]["id"]
deployment_details = wml_client.deployments.create(artifact_uid=model_id, meta_props=metadata)